<a href="https://colab.research.google.com/github/taniseb/megasena/blob/main/Adicionando_dados_loterias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Primeiro vamos abrir os dados que já existem

In [ ]:
from urllib.request import urlopen
import json
import ssl
import time
import urllib.error
ssl._create_default_https_context = ssl._create_unverified_context
import pandas as pd
from google.colab import files
uploaded = files.upload()

In [50]:
df = pd.read_csv('mega_sena-4.csv', index_col=0)

Repetimos as funções criadas anteriormente.

In [ ]:
def acumulado(x):
  if x == True:
    return 'SIM'
  else:
    return 'NÃO'
def muni_ganhador(x):
  if data_json['listaMunicipioUFGanhadores']==[]:
    return ""
  else:
    return data_json['listaMunicipioUFGanhadores'][0]['municipio']
  
def uf_ganhador(x):
  if data_json['listaMunicipioUFGanhadores']==[]:
    return ""
  else:
    return data_json['listaMunicipioUFGanhadores'][0]['uf']

Criamos a base a ser adicionada.


In [ ]:
base = []
dados = {}
def montar_a_base():
    dados = {
    'Concurso' : data_json['numero'],
    'Data Sorteio' : data_json['dataApuracao'],
    'Arrecadacao Total': data_json['valorArrecadado'],
    'Dezena_1': data_json['dezenasSorteadasOrdemSorteio'][0],
    'Dezena_2': data_json['dezenasSorteadasOrdemSorteio'][1],
    'Dezena_3': data_json['dezenasSorteadasOrdemSorteio'][2],
    'Dezena_4': data_json['dezenasSorteadasOrdemSorteio'][3],
    'Dezena_5': data_json['dezenasSorteadasOrdemSorteio'][4],
    'Dezena_6': data_json['dezenasSorteadasOrdemSorteio'][5],
    'Ganhadores Sena': data_json['listaRateioPremio'][0]['numeroDeGanhadores'],
    'Rateio Sena': data_json['listaRateioPremio'][0]['valorPremio'],
    'Cidade': muni_ganhador(data_json['listaRateioPremio'][0]['numeroDeGanhadores']),
    'UF': uf_ganhador(data_json['listaRateioPremio'][0]['numeroDeGanhadores']),
    'Ganhadores Quina': data_json['listaRateioPremio'][1]['numeroDeGanhadores'],
    'Rateio Quina': data_json['listaRateioPremio'][1]['valorPremio'],
    'Ganhadores Quadra': data_json['listaRateioPremio'][2]['numeroDeGanhadores'],
    'Rateio Quadra': data_json['listaRateioPremio'][2]['valorPremio'],
    'Acumulado': acumulado(data_json['acumulado']),
    'Valor_Acumulado': data_json['valorAcumuladoProximoConcurso'],
    'Estimativa de Prêmio': data_json['valorEstimadoProximoConcurso'],
    'Acumulado Mega da Virada': data_json['valorAcumuladoConcursoEspecial'],
    'Local do Sorteio': data_json['localSorteio'],
    'Cidade UF Sorteio': data_json['nomeMunicipioUFSorteio']
    }
    base.append(dados)
    dados = {}
    return(base)



Criamos uma variável que recebe o valor do último concurso

In [ ]:
ultimo = df['Concurso'].iloc[-1]

In [ ]:
E conferimos o último concurso no site da caixa, no caso é o 2550.

In [ ]:
x = True
i = ultimo
while x == True:
  url = f"https://servicebus2.caixa.gov.br/portaldeloterias/api/megasena/{i+1}"
  try:
    print(i)
    response = urlopen(url)
    data_json = json.loads(response.read())
    montar_a_base()
    i+=1
    if i > 2549:
      x = False
  except urllib.error.HTTPError as exc:
    print('Error code: ', exc)
    time.sleep(10) # wait 10 seconds and then make http request again
    continue
  

2549


In [ ]:
base

[{'Concurso': 2550,
  'Data Sorteio': '31/12/2022',
  'Arrecadacao Total': 1958449342.5,
  'Dezena_1': '59',
  'Dezena_2': '34',
  'Dezena_3': '04',
  'Dezena_4': '58',
  'Dezena_5': '10',
  'Dezena_6': '05',
  'Ganhadores Sena': 5,
  'Rateio Sena': 108393993.26,
  'Cidade': 'CANAL ELETRONICO',
  'UF': '--',
  'Ganhadores Quina': 2485,
  'Rateio Quina': 45438.78,
  'Ganhadores Quadra': 183921,
  'Rateio Quadra': 877.04,
  'Acumulado': 'NÃO',
  'Valor_Acumulado': 0.0,
  'Estimativa de Prêmio': 3000000.0,
  'Acumulado Mega da Virada': 0.0,
  'Local do Sorteio': 'ESTÚDIO REDE GLOBO',
  'Cidade UF Sorteio': 'SÃO PAULO,  SP'}]

In [51]:
df_atualizada = df.append(base)

e salvamos novamente os resultados

In [46]:
df = pd.DataFrame(df_atualizada)

In [53]:
df_atualizada.to_csv("mega_sena.csv", encoding= 'utf_8')